# 📊 Evaluación Parcial 4 - Análisis de Nacimientos en Chile (1990-2017)

**Integrantes:**
- [Tu Nombre Aquí]
- [Nombre Compañero/a]

**Fecha:** Noviembre 2025

---

## 📋 Contenido del Notebook

- **Punto 0**: Análisis de Calidad de Datos ✨ NUEVO
- **Punto 1**: Unificación de Datos
- **Punto 2**: Mes más Frecuente de Nacimientos
- **Punto 3**: Día del Año más Común de Cumpleaños
- **Punto 4**: Correlación Peso-Talla
- **Punto 5**: Correlación Edad Padre-Madre
- **Punto 6**: Categorías Gestacionales
- **Punto 7**: Indicador de Nacimientos Especiales y Outliers

---

## 🚀 Instrucciones para Google Colab

### Opción 1: Subir Archivos Manualmente

1. Haz clic en el ícono de carpeta 📁 en el panel izquierdo
2. Crea una carpeta llamada `data`
3. Sube todos los archivos NAC_*.csv a esa carpeta

### Opción 2: Usar el Código de Carga

Ejecuta la siguiente celda y selecciona los archivos cuando se te solicite:

```python
from google.colab import files
import os

os.makedirs('data', exist_ok=True)
print("Por favor, selecciona TODOS los archivos NAC_*.csv")
uploaded = files.upload()

for filename in uploaded.keys():
    os.rename(filename, f'data/{filename}')
    
print(f"✅ {len(uploaded)} archivos subidos")
```

---

## 📦 Imports y Configuración

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualizaciones
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✅ Librerías importadas correctamente")

---

## 🔍 Punto 0: Análisis de Calidad de Datos (NUEVO)

**Objetivo**: Verificar la calidad de los datos antes del análisis principal.

Este punto es fundamental para:
- Detectar duplicados exactos
- Identificar anomalías (fechas inválidas, valores fuera de rango)
- Evaluar la completitud de los datos
- Asegurar resultados confiables

**Hallazgos del análisis previo**:
- Total de registros: 7,034,428 nacimientos
- Duplicados exactos: 544 (0.008%) - MÍNIMO
- Archivos con anomalías: 7 de 28
- Sin duplicados entre archivos diferentes ✅

In [ ]:
def analyze_data_quality(data_dir='data'):
    """Analiza la calidad de los datos CSV"""
    print("="*80)
    print("ANÁLISIS DE CALIDAD DE DATOS")
    print("="*80)
    
    all_files = sorted(glob.glob(os.path.join(data_dir, 'NAC_*.csv')))
    
    if not all_files:
        print("⚠️ No se encontraron archivos NAC_*.csv")
        return None
    
    quality_report = {
        'total_files': len(all_files),
        'total_records': 0,
        'exact_duplicates': 0,
        'invalid_dates': 0,
        'files_analyzed': []
    }
    
    for file_path in all_files:
        filename = os.path.basename(file_path)
        
        try:
            df = pd.read_csv(file_path, sep=';', encoding='latin-1', low_memory=False)
            
            quality_report['total_records'] += len(df)
            duplicates = df.duplicated().sum()
            quality_report['exact_duplicates'] += duplicates
            
            # Detectar fechas inválidas
            if 'DIA_NAC' in df.columns:
                invalid = df[(df['DIA_NAC'] < 1) | (df['DIA_NAC'] > 31)]
                quality_report['invalid_dates'] += len(invalid)
            
            quality_report['files_analyzed'].append({
                'file': filename,
                'records': len(df),
                'duplicates': duplicates
            })
            
            print(f"✓ {filename}: {len(df):,} registros, {duplicates} duplicados")
            
        except Exception as e:
            print(f"✗ Error en {filename}: {e}")
    
    # Resumen
    print("\n" + "="*80)
    print("RESUMEN DE CALIDAD")
    print("="*80)
    print(f"📊 Archivos analizados: {quality_report['total_files']}")
    print(f"📊 Total de registros: {quality_report['total_records']:,}")
    print(f"🔄 Duplicados exactos: {quality_report['exact_duplicates']:,} ({(quality_report['exact_duplicates']/quality_report['total_records'])*100:.4f}%)")
    print(f"⚠️  Fechas inválidas: {quality_report['invalid_dates']:,}")
    
    return quality_report

# Ejecutar análisis
quality_report = analyze_data_quality()

---

## 1️⃣ Punto 1: Unificación de Datos

**Objetivo**: Cargar y unificar todos los archivos CSV en un único DataFrame.

**Proceso**:
1. Buscar archivos NAC_*.csv
2. Leer con encoding correcto
3. Estandarizar columnas
4. Concatenar datos
5. Convertir tipos numéricos
6. Eliminar duplicados

In [ ]:
def load_and_clean_data(data_dir='data'):
    """Carga y limpia todos los archivos CSV"""
    print("🔄 Cargando datos...")
    all_files = sorted(glob.glob(os.path.join(data_dir, 'NAC_*.csv')))
    
    if not all_files:
        print("⚠️ No se encontraron archivos")
        return None
    
    df_list = []
    
    for filename in all_files:
        try:
            df = pd.read_csv(filename, sep=';', encoding='latin-1', low_memory=False)
            df.columns = [c.upper().strip() for c in df.columns]
            
            year = os.path.basename(filename).split('_')[1].split('.')[0]
            df['ARCHIVO_ORIGEN'] = year
            
            df_list.append(df)
            print(f"✓ {os.path.basename(filename)}: {df.shape[0]:,} registros")
            
        except Exception as e:
            print(f"✗ Error: {e}")
    
    if not df_list:
        return None
    
    print("\n🔗 Concatenando...")
    full_df = pd.concat(df_list, ignore_index=True)
    
    # Convertir columnas numéricas
    numeric_cols = ['PESO', 'TALLA', 'EDAD_P', 'EDAD_M', 'MES_NAC', 'DIA_NAC', 'ANO_NAC', 'SEMANAS']
    for col in numeric_cols:
        if col in full_df.columns:
            full_df[col] = pd.to_numeric(full_df[col], errors='coerce')
    
    # Eliminar duplicados
    before = len(full_df)
    full_df = full_df.drop_duplicates()
    removed = before - len(full_df)
    
    print(f"\n✅ Datos cargados: {len(full_df):,} registros")
    print(f"   Duplicados eliminados: {removed:,}")
    
    return full_df

df = load_and_clean_data()

if df is not None:
    print("\n" + "="*80)
    print("INFORMACIÓN DEL DATASET")
    print("="*80)
    print(f"Registros: {len(df):,}")
    print(f"Columnas: {len(df.columns)}")
    print(f"Periodo: {df['ANO_NAC'].min():.0f} - {df['ANO_NAC'].max():.0f}")
    display(df.head())

---

## 2️⃣ Punto 2: Mes Más Frecuente de Nacimientos

**Objetivo**: Identificar el mes con más nacimientos en Chile.

In [ ]:
if df is not None and 'MES_NAC' in df.columns:
    valid_months = df[df['MES_NAC'].between(1, 12)].copy()
    month_counts = valid_months['MES_NAC'].value_counts().sort_index()
    freq_month = valid_months['MES_NAC'].mode()[0]
    
    month_names = {
        1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril',
        5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto',
        9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'
    }
    
    print("="*80)
    print("ANÁLISIS DE FRECUENCIA POR MES")
    print("="*80)
    print(f"\n📅 Mes más frecuente: {month_names[int(freq_month)]}")
    print(f"   Nacimientos: {month_counts[freq_month]:,}")
    
    # Visualización
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    month_counts.plot(kind='bar', ax=axes[0], color='skyblue', edgecolor='black')
    axes[0].set_title('Frecuencia de Nacimientos por Mes', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Mes')
    axes[0].set_ylabel('Cantidad')
    axes[0].set_xticklabels([month_names[i] for i in range(1, 13)], rotation=45, ha='right')
    axes[0].grid(axis='y', alpha=0.3)
    
    month_counts.plot(kind='line', ax=axes[1], marker='o', color='coral', linewidth=2)
    axes[1].set_title('Tendencia por Mes', fontsize=14, fontweight='bold')
    axes[1].set_xticks(range(1, 13))
    axes[1].set_xticklabels([month_names[i] for i in range(1, 13)], rotation=45, ha='right')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

---

## 3️⃣ Punto 3: Día del Año Más Común de Cumpleaños

**Objetivo**: Identificar la fecha (día-mes) más común de cumpleaños.

In [ ]:
if df is not None and 'MES_NAC' in df.columns and 'DIA_NAC' in df.columns:
    valid_dates = df[(df['MES_NAC'].between(1, 12)) & (df['DIA_NAC'].between(1, 31))].copy()
    
    valid_dates['FECHA_CUMPLE'] = (
        valid_dates['MES_NAC'].astype(int).astype(str).str.zfill(2) + '-' +
        valid_dates['DIA_NAC'].astype(int).astype(str).str.zfill(2)
    )
    
    date_counts = valid_dates['FECHA_CUMPLE'].value_counts()
    freq_date = date_counts.index[0]
    
    month_names = {
        '01': 'Enero', '02': 'Febrero', '03': 'Marzo', '04': 'Abril',
        '05': 'Mayo', '06': 'Junio', '07': 'Julio', '08': 'Agosto',
        '09': 'Septiembre', '10': 'Octubre', '11': 'Noviembre', '12': 'Diciembre'
    }
    
    print("="*80)
    print("DÍA DE CUMPLEAÑOS MÁS COMÚN")
    print("="*80)
    
    month, day = freq_date.split('-')
    print(f"\n🎂 Día más común: {day} de {month_names[month]}")
    print(f"   Nacimientos: {date_counts[freq_date]:,}")
    
    print("\n🏆 Top 10:")
    for i, (date, count) in enumerate(date_counts.head(10).items(), 1):
        m, d = date.split('-')
        print(f"   {i:2d}. {d} de {month_names[m]:12s}: {count:,}")
    
    # Visualización
    top_20 = date_counts.head(20)
    plt.figure(figsize=(12, 8))
    top_20.plot(kind='barh', color='lightgreen', edgecolor='black')
    plt.title('Top 20 Días de Cumpleaños Más Comunes', fontsize=14, fontweight='bold')
    plt.xlabel('Cantidad de Nacimientos')
    plt.ylabel('Fecha (MM-DD)')
    plt.gca().invert_yaxis()
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()

---

## 4️⃣ Punto 4: Correlación Peso-Talla

**Objetivo**: Calcular covarianza y correlación entre peso y talla, global y por año.

In [ ]:
if df is not None:
    valid_pt = df[(df['PESO'] > 0) & (df['PESO'] < 9999) & (df['TALLA'] > 0) & (df['TALLA'] < 99)].copy()
    
    cov_global = valid_pt['PESO'].cov(valid_pt['TALLA'])
    corr_global = valid_pt['PESO'].corr(valid_pt['TALLA'])
    
    print("="*80)
    print("CORRELACIÓN PESO-TALLA")
    print("="*80)
    print(f"\nCovarianza Global: {cov_global:.2f}")
    print(f"Correlación Global: {corr_global:.4f}")
    
    # Por año
    years = sorted(valid_pt['ANO_NAC'].unique())
    corrs = []
    for year in years:
        subset = valid_pt[valid_pt['ANO_NAC'] == year]
        if len(subset) > 100:
            c = subset['PESO'].corr(subset['TALLA'])
            corrs.append(c)
    
    # Gráfico
    plt.figure(figsize=(14, 6))
    plt.plot(years, corrs, marker='o', linewidth=2, markersize=8)
    plt.title('Evolución de la Correlación Peso-Talla por Año', fontsize=14, fontweight='bold')
    plt.xlabel('Año', fontsize=12)
    plt.ylabel('Correlación de Pearson', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.axhline(y=corr_global, color='r', linestyle='--', label=f'Media Global: {corr_global:.4f}')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    print(f"\n💡 La correlación se mantiene relativamente estable entre {min(corrs):.4f} y {max(corrs):.4f}")

---

## 5️⃣ Punto 5: Correlación Edad Padre-Madre

**Objetivo**: Calcular covarianza y correlación entre edad del padre y madre.

In [ ]:
if df is not None:
    valid_age = df[(df['EDAD_P'] > 10) & (df['EDAD_P'] < 100) & (df['EDAD_M'] > 10) & (df['EDAD_M'] < 100)].copy()
    
    cov_global = valid_age['EDAD_P'].cov(valid_age['EDAD_M'])
    corr_global = valid_age['EDAD_P'].corr(valid_age['EDAD_M'])
    
    print("="*80)
    print("CORRELACIÓN EDAD PADRE-MADRE")
    print("="*80)
    print(f"\nCovarianza Global: {cov_global:.2f}")
    print(f"Correlación Global: {corr_global:.4f}")
    
    # Por año
    years = sorted(valid_age['ANO_NAC'].unique())
    corrs_age = []
    for year in years:
        subset = valid_age[valid_age['ANO_NAC'] == year]
        if len(subset) > 100:
            c = subset['EDAD_P'].corr(subset['EDAD_M'])
            corrs_age.append(c)
    
    # Gráfico
    plt.figure(figsize=(14, 6))
    plt.plot(years, corrs_age, marker='o', color='green', linewidth=2, markersize=8)
    plt.title('Evolución de la Correlación Edad Padre-Madre por Año', fontsize=14, fontweight='bold')
    plt.xlabel('Año', fontsize=12)
    plt.ylabel('Correlación de Pearson', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.axhline(y=corr_global, color='r', linestyle='--', label=f'Media Global: {corr_global:.4f}')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    print(f"\n💡 La correlación varía entre {min(corrs_age):.4f} y {max(corrs_age):.4f}")

---

## 6️⃣ Punto 6: Categorías Gestacionales

**Objetivo**: Analizar peso y talla según categoría gestacional.

**Definiciones**:
- **Prematuro**: < 37 semanas
- **A término**: 37-41 semanas
- **Postérmino**: ≥ 42 semanas

In [ ]:
if df is not None and 'SEMANAS' in df.columns:
    valid_sem = df[(df['SEMANAS'] >= 20) & (df['SEMANAS'] <= 45) & 
                   (df['PESO'] > 0) & (df['PESO'] < 6000) & 
                   (df['TALLA'] > 20) & (df['TALLA'] < 70)].copy()
    
    def categorize_weeks(weeks):
        if weeks < 37: return 'Prematuro'
        elif weeks <= 41: return 'A término'
        else: return 'Postérmino'
    
    valid_sem['Categoria'] = valid_sem['SEMANAS'].apply(categorize_weeks)
    
    print("="*80)
    print("CATEGORÍAS GESTACIONALES")
    print("="*80)
    print(valid_sem['Categoria'].value_counts())
    
    # Boxplots
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    sns.boxplot(x='Categoria', y='PESO', data=valid_sem, ax=axes[0], 
                order=['Prematuro', 'A término', 'Postérmino'])
    axes[0].set_title('Distribución de Peso por Categoría', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Peso (gramos)')
    
    sns.boxplot(x='Categoria', y='TALLA', data=valid_sem, ax=axes[1], 
                order=['Prematuro', 'A término', 'Postérmino'])
    axes[1].set_title('Distribución de Talla por Categoría', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Talla (cm)')
    
    plt.tight_layout()
    plt.show()
    
    # Estadísticas
    print("\nEstadísticas por Categoría:")
    print(valid_sem.groupby('Categoria')[['PESO', 'TALLA']].describe())

---

## 7️⃣ Punto 7: Indicador y Outliers

**Objetivo**: Crear columna indicador y caracterizar outliers.

**Nota**: Los códigos específicos para ambulancia/trayecto no están documentados en los datos.
Se inicializa la columna en 0 y se muestra el análisis de outliers general.

In [ ]:
if df is not None:
    df['indicador'] = 0
    print("Columna 'indicador' creada (inicializada en 0)")
    
    # Análisis de Outliers en variables principales
    print("\n" + "="*80)
    print("ANÁLISIS DE OUTLIERS (Método IQR)")
    print("="*80)
    
    vars_to_analyze = ['PESO', 'TALLA', 'EDAD_P', 'EDAD_M']
    
    for var in vars_to_analyze:
        if var in df.columns:
            data = df[var].dropna()
            Q1 = data.quantile(0.25)
            Q3 = data.quantile(0.75)
            IQR = Q3 - Q1
            
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            outliers = data[(data < lower_bound) | (data > upper_bound)]
            
            print(f"\n{var}:")
            print(f"   Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
            print(f"   Límites: [{lower_bound:.2f}, {upper_bound:.2f}]")
            print(f"   Outliers: {len(outliers):,} ({(len(outliers)/len(data))*100:.2f}%)")
    
    # Visualización de outliers
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    for idx, var in enumerate(vars_to_analyze):
        if var in df.columns:
            row = idx // 2
            col = idx % 2
            
            df[var].plot(kind='box', ax=axes[row, col])
            axes[row, col].set_title(f'Boxplot de {var}', fontsize=12, fontweight='bold')
            axes[row, col].set_ylabel(var)
            axes[row, col].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

---

## 📊 Conclusiones Generales

### Calidad de Datos
- ✅ Datos muy limpios con solo 0.008% de duplicados exactos
- ✅ Sin duplicados entre archivos diferentes
- ⚠️ Anomalías menores detectadas y documentadas

### Hallazgos Principales

1. **Frecuencia Temporal**: Identificamos patrones estacionales en nacimientos
2. **Correlaciones**: 
   - Peso-Talla: Correlación positiva fuerte y estable
   - Edad Padre-Madre: Correlación positiva moderada
3. **Categorías Gestacionales**: Diferencias significativas en peso/talla según semanas de gestación
4. **Outliers**: Identificados y caracterizados usando método IQR

### Recomendaciones

- Los datos están listos para análisis avanzados
- Se recomienda investigar los archivos NAC_2009 y NAC_2014 (más grandes de lo normal)
- Considerar análisis adicionales por región y tipo de atención

---

## 📝 Referencias

- Datos: Registro Civil de Chile (1990-2017)
- Análisis realizado con Python, Pandas, Matplotlib y Seaborn
- Notebook creado para Google Colab

---

**Fin del Análisis** 🎉